In [21]:
from fairlearn.metrics import MetricFrame, selection_rate, false_positive_rate, false_negative_rate,true_negative_rate,true_positive_rate,demographic_parity_difference,equalized_odds_difference
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
import pandas as pd
import joblib


In [22]:
#Loading the model and saved data
model = joblib.load("model/randomforest_model.pkl")
df = pd.read_csv("resume_cleaned.csv")

X=df.drop(columns=["received_callback"])
y=df["received_callback"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) 

# Predicting again using same trained model
y_pred = model.predict(X_test)

Run this for Fairness metrics by gender

In [25]:
#set protected attribute to gender
protected = df.loc[X_test.index,'gender']

Run this for Fairness metrics by race

In [23]:
#set protected attribute to race
protected = df.loc[X_test.index,'race']

In [26]:
#Fairness metrics after one of the protected attributes are set
metric_frame = MetricFrame(
  metrics={
    "accuracy":accuracy_score,
    "precision":precision_score,
    "recall":recall_score,
    "f1_score":f1_score,
    "selection_rate":selection_rate
  },
  y_true=y_test,
  y_pred=y_pred,
  sensitive_features=protected
)

print("Groupwise Fairness Metrics:\n")
print(metric_frame.by_group)

Groupwise Fairness Metrics:

        accuracy  precision    recall  f1_score  selection_rate
gender                                                         
0       0.898551   0.055556  0.016393  0.025316        0.023715
1       0.916279   0.333333  0.058824  0.100000        0.013953


Run this to compare gender vs race (best)

In [27]:
for attr in ['gender', 'race']:
    print(f"\n--- Fairness metrics by {attr} ---")
    protected = df.loc[X_test.index, attr]

    metric_frame = MetricFrame(
        metrics={
            "accuracy": accuracy_score,
            "precision": precision_score,
            "recall": recall_score,
            "f1_score":f1_score,
            "selection_rate": selection_rate
        },
        y_true=y_test,
        y_pred=model.predict(X_test),
        sensitive_features=protected
    )
    
    print(metric_frame.by_group)


--- Fairness metrics by gender ---
        accuracy  precision    recall  f1_score  selection_rate
gender                                                         
0       0.898551   0.055556  0.016393  0.025316        0.023715
1       0.916279   0.333333  0.058824  0.100000        0.013953

--- Fairness metrics by race ---
      accuracy  precision    recall  f1_score  selection_rate
race                                                         
0     0.923529   0.111111  0.031250  0.048780        0.017647
1     0.879310   0.083333  0.021739  0.034483        0.025862


In [28]:
from fairlearn.metrics import(
  demographic_parity_difference,
  demographic_parity_ratio,
  equalized_odds_difference,
  equalized_odds_ratio
)
print(str(protected)) #just to see if gender or race is chosen

#Statistical Parity Difference
spd = demographic_parity_difference(y_test, y_pred, sensitive_features=protected)

#Disparate Impact (selection rate ratio)
di = demographic_parity_ratio(y_test, y_pred, sensitive_features=protected)

#Equal Opportunity Difference (TPR diff)
eod = equalized_odds_difference(y_test, y_pred, sensitive_features=protected)

print(f"Statistical Parity Difference: {spd:.4f}")
print(f"Disparate Impact: {di:.4f}")
print(f"Equal Opportunity Difference: {eod:.4f}")

2027    0
509     1
4474    1
3061    0
603     0
       ..
3242    1
3407    0
3710    1
3303    1
4187    0
Name: race, Length: 974, dtype: int64
Statistical Parity Difference: 0.0082
Disparate Impact: 0.6824
Equal Opportunity Difference: 0.0096


BIAS MITIGATION TECHNIQUES

- ExponentiatedGradient with the DemographicParity

In [29]:
from fairlearn.reductions import ExponentiatedGradient, DemographicParity
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [30]:
protected = df.loc[X_test.index,'gender']

In [11]:
protected = df.loc[X_test.index,'race']

- Fairness Constraint model training

In [31]:
base_model = RandomForestClassifier(random_state=42)
constraint = DemographicParity()
mitigator = ExponentiatedGradient(estimator=base_model,constraints=constraint)


In [32]:
mitigator.fit(X_train,y_train,sensitive_features=df.loc[X_train.index,'gender'])

ExponentiatedGradient(constraints=<fairlearn.reductions._moments.utility_parity.DemographicParity object at 0x0000024695CE3490>,
                      estimator=RandomForestClassifier(random_state=42),
                      nu=np.float64(0.0013211685492589627))

In [13]:
mitigator.fit(X_train,y_train,sensitive_features=df.loc[X_train.index,'race'])

ExponentiatedGradient(constraints=<fairlearn.reductions._moments.utility_parity.DemographicParity object at 0x00000272F04DA510>,
                      estimator=RandomForestClassifier(random_state=42),
                      nu=np.float64(0.0013211685492589627))

- Making predictions and Evaluations

In [36]:
y_pred_fair = mitigator.predict(X_test)
print("Accuracy after bias mitigation: ",accuracy_score(y_test,y_pred_fair))
print("Classification report: \n",classification_report(y_test,y_pred_fair))

Accuracy after bias mitigation:  0.9055441478439425
Classification report: 
               precision    recall  f1-score   support

           0       0.92      0.98      0.95       896
           1       0.11      0.03      0.04        78

    accuracy                           0.91       974
   macro avg       0.52      0.50      0.50       974
weighted avg       0.86      0.91      0.88       974



- Find fairness metrics again for new mitigated model

In [28]:
from fairlearn.metrics import MetricFrame, selection_rate, demographic_parity_difference

metric_frame_fair = MetricFrame(
    metrics={
        "accuracy": accuracy_score,
        "precision": lambda y, y_pred: precision_score(y, y_pred, zero_division=0),
        "recall": recall_score,
        "selection_rate": selection_rate
    },
    y_true=y_test,
    y_pred=y_pred_fair,
    sensitive_features=protected
)

print("\n Groupwise Fairness Metrics (Post-Mitigation):\n")
print(metric_frame_fair.by_group)

# Statistical metrics
from fairlearn.metrics import (
    demographic_parity_difference,
    demographic_parity_ratio,
    equalized_odds_difference,
    equalized_odds_ratio
)

spd = demographic_parity_difference(y_test, y_pred_fair, sensitive_features=protected)
di = demographic_parity_ratio(y_test, y_pred_fair, sensitive_features=protected)
eod = equalized_odds_difference(y_test, y_pred_fair, sensitive_features=protected)

print("\nBias Metrics (Post-Mitigation):")

print(f"Statistical Parity Difference: {spd:.4f}")
print(f"Disparate Impact: {di:.4f}")
print(f"Equal Opportunity Difference: {eod:.4f}")


 Groupwise Fairness Metrics (Post-Mitigation):

        accuracy  precision    recall  selection_rate
gender                                               
0       0.901186     0.0625  0.016393        0.021080
1       0.920930     0.5000  0.058824        0.009302

Bias Metrics (Post-Mitigation):
Statistical Parity Difference: 0.0118
Disparate Impact: 0.4413
Equal Opportunity Difference: 0.0424


************************************************************************

- ExponentiatedGradient with the EqualizedOdds (only for gender)

In [3]:
from fairlearn.reductions import ExponentiatedGradient, DemographicParity
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
import joblib
import numpy as np

# Load model data
X_train = joblib.load("orgmodel/X_train.pkl")
y_train = joblib.load("orgmodel/y_train.pkl")
X_test = joblib.load("orgmodel/X_test.pkl")
y_test = joblib.load("orgmodel/y_test.pkl")

# Load full DataFrame to get protected attribute
df = pd.read_csv("resume_cleaned.csv")

# Ensure correct index alignment
protected_train = df.loc[X_train.index, 'gender']
protected_test = df.loc[X_test.index, 'gender']

# Define base estimator with balanced class weights
base_estimator = RandomForestClassifier(class_weight='balanced')

# Initialize fairness-aware mitigation using Demographic Parity
mitigator = ExponentiatedGradient(
    estimator=base_estimator,
    constraints=DemographicParity()
)

# Fit the mitigation model
mitigator.fit(X_train, y_train, sensitive_features=protected_train)

# Predict
preds = mitigator.predict(X_test)

# Check if model is predicting both classes
print("Unique Predictions:", np.unique(preds, return_counts=True))

# Evaluate accuracy
acc = accuracy_score(y_test, preds)
print(f"\nAccuracy: {acc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, preds))

# Grouped fairness metrics
group_metrics = pd.DataFrame({
    "accuracy": y_test == preds,
    "prediction": preds,
    "actual": y_test,
    "group": protected_test
}).groupby("group").agg(
    accuracy=("accuracy", "mean"),
    precision=("prediction", lambda x: np.mean((x == 1))),
    recall=("actual", lambda x: np.mean((x == 1))),
    selection_rate=("prediction", lambda x: np.mean(x))
)

print("\nGroupwise Fairness Metrics (Post-Mitigation):")
print(group_metrics)

# Bias Metrics
def calculate_bias_metrics(y_true, y_pred, protected):
    group_0 = protected == 0
    group_1 = protected == 1

    # Selection rates
    sr0 = np.mean(y_pred[group_0])
    sr1 = np.mean(y_pred[group_1])

    # TPR (Recall)
    tpr0 = np.mean(y_pred[group_0][y_true[group_0] == 1])
    tpr1 = np.mean(y_pred[group_1][y_true[group_1] == 1])

    spd = sr1 - sr0
    di = sr1 / sr0 if sr0 != 0 else float('nan')
    eod = tpr1 - tpr0

    return spd, di, eod

spd, di, eod = calculate_bias_metrics(y_test.values, preds, protected_test.values)
print("\nBias Metrics (Post-Mitigation):")
print(f"Statistical Parity Difference: {spd:.4f}")
print(f"Disparate Impact: {di:.4f}")
print(f"Equal Opportunity Difference: {eod:.4f}")


Unique Predictions: (array([0, 1]), array([911,  63]))

Accuracy: 0.8758

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.94      0.93       896
           1       0.16      0.13      0.14        78

    accuracy                           0.88       974
   macro avg       0.54      0.53      0.54       974
weighted avg       0.86      0.88      0.87       974


Groupwise Fairness Metrics (Post-Mitigation):
       accuracy  precision    recall  selection_rate
group                                               
0      0.874835   0.068511  0.080369        0.068511
1      0.879070   0.051163  0.079070        0.051163

Bias Metrics (Post-Mitigation):
Statistical Parity Difference: -0.0173
Disparate Impact: 0.7468
Equal Opportunity Difference: -0.0887
